# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.1. [**Validation croisée : évaluer les performances des estimateurs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-evaluating-estimator-performance)<br/>([_Cross-validation: evaluating estimator performance_](https://scikit-learn.org/stable/model_selection.html#cross-validation-evaluating-estimator-performance))

# Sommaire

- **Volume** : 25 pages, 7 exemples, 8 papiers
- 3.1.1. [**Calcul de métriques à validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#computing-cross-validated-metrics)<br/>([_computing-cross-validated-metrics_](https://scikit-learn.org/stable/model_selection.html#computing-cross-validated-metrics))
- 3.1.2. [**Itérateurs de validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-iterators)<br/>([_Cross validation iterators_](https://scikit-learn.org/stable/model_selection.html#cross-validation-iterators))
- 3.1.3. [**Une note sur le brassage**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#a-note-on-shuffling)<br/>([_A note on shuffling_](https://scikit-learn.org/stable/model_selection.html#a-note-on-shuffling))
- 3.1.4. [**Validation croisée et sélection de modèles**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#cross-validation-and-model-selection)<br/>([_cross-validation-and-model-selection_](https://scikit-learn.org/stable/model_selection.html#cross-validation-and-model-selection))
- 3.1.5. [**Score de test de permutation**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#permutation-test-score)<br/>([_Permutation test score_](https://scikit-learn.org/stable/model_selection.html#permutation-test-score))

# <a id='computing-cross-validated-metrics'></a> 3.1. Validation croisée : évaluer les performances des estimateurs

Apprendre les paramètres d'une fonction de prédiction et la tester sur les mêmes données est une erreur méthodologique : un modèle qui se contenterait de répéter les étiquettes des échantillons qu'il vient de voir aurait un score parfait mais ne parviendrait pas à prédire quoi que ce soit d'utile sur de nouvelles données. Cette situation s'appelle le **surajustement**. Pour l'éviter, il est courant lors de la réalisation d'une expérience d'apprentissage automatique (supervisée) de conserver une partie des données disponibles sous forme d'**ensemble de test** `X_test`, `y_test`. Notez que le mot "expérience" n'est pas destiné à désigner uniquement un usage académique, car même dans les environnements commerciaux, l'apprentissage automatique commence généralement de manière expérimentale. Voici un organigramme du flux de travail typique de validation croisée dans l'entraînement de modèles. Les meilleurs paramètres peuvent être déterminés par des techniques comme la [**recherche en grille** (3.2.1)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search).


<div style="background-color: white; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/grid_search_workflow.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

Dans scikit-learn, une répartition aléatoire en ensembles d'apprentissage et de test peut être rapidement calculée avec la fonction utilitaire [**`train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split). Chargeons l'ensemble de données Iris pour y adapter une machine à vecteurs de support linéaire :

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

((150, 4), (150,))

Nous pouvons désormais échantillonner rapidement un ensemble d'entraînement tout en conservant 40 % des données pour tester (évaluer) notre classifieur :

In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

X_train.shape, y_train.shape
# ((90, 4), (90,))
X_test.shape, y_test.shape
# ((60, 4), (60,))

clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
clf.score(X_test, y_test)
# 0.96...

0.9666666666666667

Lors de l'évaluation de différents paramétrages ("hyperparamètres") pour les estimateurs, tels que le paramètre `C` qui doit être défini manuellement pour une SVM, il existe toujours un risque de surapprentissage sur l'_ensemble de test_ car les paramètres peuvent être modifiés jusqu'à ce que l'estimateur fonctionne de manière optimale. De cette façon, les connaissances sur l'ensemble de test peuvent "fuir" dans le modèle et les métriques d'évaluation ne plus rendre compte des performances de généralisation. Pour résoudre ce problème, une autre partie de l'ensemble de données peut être isolée en tant qu'"ensemble de validation": l'entraînement se déroule sur l'ensemble d'entraînement, après quoi l'évaluation est effectuée sur l'ensemble de validation, et quand l'expérience semble être réussie, l'évaluation finale peut être effectuée sur l'ensemble de test.

Cependant, en partitionnant les données disponibles en trois ensembles, nous réduisons considérablement le nombre d'échantillons qui peuvent être utilisés pour entraîner le modèle, et les résultats peuvent dépendre d'un choix aléatoire particulier pour la paire d'ensembles (entraînement, validation).

Une solution à ce problème est une procédure appelée [⦿ **validation croisée**](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) (CV (_Cross validation_) en abrégé). Un ensemble de test doit toujours être conservé pour l'évaluation finale, mais l'ensemble de validation n'est plus nécessaire lors de la réalisation de la CV. Dans l'approche de base, appelée $k$-fold CV, l'ensemble d'apprentissage est divisé en $k$ ensembles plus petits (d'autres approches sont décrites ci-dessous, mais suivent généralement les mêmes principes). La procédure suivante est suivie pour chacun des $k$ "plis" :

- Un modèle est entraîné sur $k - 1$ plis en tant que données d'apprentissage ;
- le modèle résultant est validé sur le pli restant (il est utilisé comme ensemble de test pour calculer une mesure de performance telle que l'exactitude).

La mesure de performance rapportée par la validation croisée $k$-plis est alors la moyenne des valeurs calculées dans la boucle. Cette approche peut être coûteuse en calcul, mais ne gaspille pas trop de données (comme c'est le cas lors du choix d'un ensemble de validation arbitraire), ce qui est un avantage majeur dans des problèmes tels que l'inférence inverse où le nombre d'échantillons est très petit.

<div style="background-color: white; text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

## <a id='computing-cross-validated-metrics'></a> 3.1.1. Calcul de métriques à validation croisée

La manière la plus simple d'utiliser la validation croisée consiste à appeler la fonction utilitaire [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) sur l'estimateur et l'ensemble de données.

L'exemple suivant montre comment estimer la précision d'une machine à vecteurs de support à noyau linéaire sur l'ensemble de données iris en divisant les données, en ajustant un modèle et en calculant le score 5 fois consécutives (avec des divisions différentes à chaque fois) :

In [3]:
from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X, y, cv=5)
scores
# array([0.96..., 1. , 0.96..., 0.96..., 1. ])

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

Le score moyen et l'écart type sont alors donnés par :

In [4]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# 0.98 accuracy with a standard deviation of 0.02

0.98 accuracy with a standard deviation of 0.02


Par défaut, le score calculé à chaque itération de la CV est donné par la méthode `score` de l'estimateur. Il est possible de changer cela en utilisant le paramètre `scoring`` :

In [ ]:
from sklearn import metrics
scores = cross_val_score(clf, X, y, cv=5, scoring='f1_macro')
scores
# array([0.96..., 1.  ..., 0.96..., 0.96..., 1.        ])

array([0.96658312, 1.        , 0.96658312, 0.96658312, 1.        ])

Voir [**Le paramètre `scoring` : définir les règles d'évaluation du modèle** (3.3.1)](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) pour plus de détails. Dans le cas de l'ensemble de données Iris, les échantillons sont équilibrés entre les classes cibles, d'où le fait que l'exactitude et le score F1 sont presque égaux.

Lorsque l'argument `cv` est un entier, [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) utilise par défaut les stratégies [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) ou [**`StratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html), cette dernière étant utilisée si l'estimateur dérive de [**`ClassifierMixin`**](https://scikit-learn.org/stable/modules/generated/sklearn.base.ClassifierMixin.html).

Il est également possible d'utiliser d'autres stratégies de validation croisée en passant un itérateur de validation croisée à la place, par exemple :

In [5]:
from sklearn.model_selection import ShuffleSplit
n_samples = X.shape[0]
cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
cross_val_score(clf, X, y, cv=cv)
# array([0.977..., 0.977..., 1.  ..., 0.955..., 1.        ])

array([0.97777778, 0.97777778, 1.        , 0.95555556, 1.        ])

Une autre option consiste à utiliser un itérable fournissant des séparations (entraînement, test) sous forme de tableaux d'indices, par exemple :

In [ ]:
def custom_cv_2folds(X):
    n = X.shape[0]
    for i in range(1, 3):
        idx = np.arange(n * (i - 1) / 2, n * i / 2, dtype=int)
        yield idx, idx

custom_cv = custom_cv_2folds(X)
cross_val_score(clf, X, y, cv=custom_cv)
# array([1.        , 0.973...])

array([1.        , 0.97333333])

**Transformation de données avec des données conservées**

Tout comme il est important de tester un estimateur sur les données d'entraînement conservées, le prétraitement (comme la normalisation, la sélection de caractéristiques, etc.) et les [**Transformations de données** (6)](https://scikit-learn.org/stable/data_transforms.html#data-transforms) similaires doivent également être apprises à partir d'un ensemble d'entraînement puis appliquées aux données conservées pour la prédiction :

In [6]:
from sklearn import preprocessing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_transformed = scaler.transform(X_train)
clf = svm.SVC(C=1).fit(X_train_transformed, y_train)
X_test_transformed = scaler.transform(X_test)
clf.score(X_test_transformed, y_test)
# 0.9333...

0.9333333333333333

Un [**`Pipeline`**](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) facilite la composition des estimateurs, offrant ce comportement sous validation croisée :

In [8]:
from sklearn.pipeline import make_pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
cross_val_score(clf, X, y, cv=cv)
# array([0.977..., 0.933..., 0.955..., 0.933..., 0.977...])

array([0.97777778, 0.93333333, 0.95555556, 0.93333333, 0.97777778])

Voir [**Pipelines et estimateurs composites** (6.1)](https://scikit-learn.org/stable/modules/compose.html#combining-estimators).

### <a id='the-cross-validate-function-and-multiple-metric-evaluation'></a> 3.1.1.1. La fonction `cross_validate` et l'évaluation de métriques multiples

La fonction [**`cross_validate`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html) diffère de [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) de deux manières :

- Elle permet de spécifier plusieurs métriques pour l'évaluation.
- Elle renvoie un dictionnaire contenant les temps d'ajustement, les temps d'évaluation (et éventuellement les scores d'entraînement, les estimateurs ajustés, les indices de la partition entraînement-test) en plus du score du test.

Pour une évaluation à métrique unique, où le paramètre de score est une chaîne, un `callable` ou `None`, les clés seront - `['test_score', 'fit_time', 'score_time']`

Et pour l'évaluation à métriques multiples, la valeur de retour est un dictionnaire avec les clés suivantes - `['test_<scorer1_name>', 'test_<scorer2_name>', 'test_<scorer...>', 'fit_time', 'score_time']`

La valeur par défaut de `return_train_score` est `False` pour économiser du temps de calcul. Pour évaluer également les scores sur l'ensemble d'entraînement, vous devez le définir sur `True`. Vous pouvez également conserver l'estimateur ajusté pour chaque ensemble d'entraînement en définissant `return_estimator=True`. De même, vous pouvez définir `return_indices=True` pour conserver les indices d'entraînement et de test utilisés pour diviser le jeu de données en ensembles d'entraînement et de test pour chaque partition CV.

Les multiples métriques peuvent être spécifiées sous forme de liste, de tuple ou d'ensemble de noms d'évaluateurs prédéfinis :

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_macro', 'recall_macro']
clf = svm.SVC(kernel='linear', C=1, random_state=0)
scores = cross_validate(clf, X, y, scoring=scoring)
sorted(scores.keys())
# ['fit_time', 'score_time', 'test_precision_macro', 'test_recall_macro']
scores['test_recall_macro']
# array([0.96..., 1.  ..., 0.96..., 0.96..., 1.        ])

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

Ou sous la forme d'un dictionnaire qui fait correspondre le nom de l'évaluateur à une fonction d'évaluation prédéfinie ou personnalisée :

In [10]:
from sklearn.metrics import make_scorer
scoring = {
    'prec_macro': 'precision_macro',
    'rec_macro': make_scorer(recall_score, average='macro')
}
scores = cross_validate(clf, X, y, scoring=scoring, cv=5, return_train_score=True)
sorted(scores.keys())
# ['fit_time', 'score_time', 'test_prec_macro', 'test_rec_macro',
#  'train_prec_macro', 'train_rec_macro']
scores['train_rec_macro']
# array([0.97..., 0.97..., 0.99..., 0.98..., 0.98...])

array([0.975     , 0.975     , 0.99166667, 0.98333333, 0.98333333])

Voici un exemple de `cross_validate` utilisant une métrique unique :

In [11]:
scores = cross_validate(clf, X, y,
                        scoring='precision_macro', cv=5,
                        return_estimator=True)
sorted(scores.keys())
# ['estimator', 'fit_time', 'score_time', 'test_score']

['estimator', 'fit_time', 'score_time', 'test_score']

### <a id='obtaining-predictions-by-cross-validation'></a> 3.1.1.2. Obtenir des prédictions par validation croisée

La fonction [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) a une interface similaire à [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), mais renvoie, pour chaque élément de l'entrée, la prédiction qui a été obtenue pour cet élément lorsqu'il était dans l'ensemble de test. Seules les stratégies de validation croisée qui affectent tous les éléments à un ensemble de test exactement une fois peuvent être utilisées (sinon, une exception est levée).

> **Avertissement** : Remarque sur l'utilisation inappropriée de [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)  
> Le résultat de [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) peut être différent de ceux obtenus avec [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) car les éléments sont regroupés différemment. La fonction [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) prend une moyenne sur les plis de validation croisée, tandis que [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) renvoie simplement les étiquettes (ou probabilités) de plusieurs modèles distincts non distingués. Ainsi, [**`cross_val_predict`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) n'est pas une mesure appropriée de l'erreur de généralisation.

**La fonction [`cross_val_predict`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html#sklearn.model_selection.cross_val_predict) convient pour :**

- La visualisation des prédictions obtenues à partir de différents modèles.
- Le mélange de modèles : lorsque les prédictions d'un estimateur supervisé sont utilisées pour entraîner un autre estimateur dans le cadre des méthodes ensemblistes.

Les itérateurs de validation croisée disponibles sont présentés dans la section suivante.

### Exemples

#### [**Caractéristique de fonctionnement du récepteur (Receiver Operating Characteristic - ROC) avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_roc_crossval.ipynb)<br/>([_Receiver Operating Characteristic (ROC) with cross validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html))

#### [**Élimination récursive des caractéristiques (RFE) avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/1_13_feature_selection/plot_rfe_with_cross_validation.ipynb)<br/>([_Recursive Feature Elimination (RFE) with cross-validation_](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_rfe_with_cross_validation.html))

#### [**Stratégie personnalisée de réajustement d'une recherche en grille avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_digits.ipynb)<br/>([_Custom refit strategy of a grid search with cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html))

#### [**Exemple de pipeline pour l'extraction et l'évaluation de caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([_Sample pipeline for text feature extraction and evaluation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))

#### [**Tracé de prédictions validées par validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_cv_predict.ipynb)<br/>([_Plotting Cross-Validated Predictions_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html))

#### [**Validation croisée imbriquée vs. non-imbriquée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_nested_cross_validation_iris.ipynb)<br/>([_Nested versus non-nested cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html))

## <a id='cross-validation-iterators'></a> 3.1.2. Itérateurs de validation croisée

Les sections suivantes répertorient les utilitaires pour générer des indices pouvant être utilisés pour diviser un ensemble de données en différents sous-ensembles selon différentes stratégies de validation croisée.

### <a id='cross-validation-iterators-for-i-i-d-data'></a> 3.1.2.1. Itérateurs de validation croisée pour les données i.i.d.

Lorsque l'on suppose que des données sont indépendantes et identiquement distribuées (i.i.d.), on fait l'hypothèse que tous les échantillons proviennent du même processus de génération et que ce processus de génération ne se souvient pas des échantillons générés précédemment.

Les validateurs croisés suivants peuvent être utilisés dans de tels cas.

> **Note** : Bien que les données i.i.d. soient une hypothèse courante en théorie de l'apprentissage automatique, elle est rarement vérifiée en pratique. Si l'on sait que les échantillons ont été générés à l'aide d'un processus dépendant du temps, il est préférable d'utiliser un [**schéma de validation croisée prenant en compte les séries temporelles** (3.1.2.6)](https://scikit-learn.org/stable/modules/cross_validation.html#timeseries-cv). De même, si l'on sait que le processus de génération présente une structure de groupe (échantillons collectés auprès de différents sujets, expériences, dispositifs de mesure), il est préférable d'utiliser [**la validation croisée par groupe** (3.1.2.3)](https://scikit-learn.org/stable/modules/cross_validation.html#group-cv).

#### <a id='k-fold'></a> K-fold

[**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) divise tous les échantillons en $k$ groupes d'échantillons, appelés plis (si $k = n$, cela équivaut à la stratégie _Leave One Out_), de tailles égales (si possible). La fonction de prédiction est entraînée à l'aide de $k - 1$ plis, et le pli laissé de côté est utilisé pour le test.

Exemple de validation croisée à 2 plis sur un ensemble de données avec 4 échantillons :

In [2]:
import numpy as np
from sklearn.model_selection import KFold

X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print(f"{train} {test}")
# [2 3] [0 1]
# [0 1] [2 3]

[2 3] [0 1]
[0 1] [2 3]


Voici une visualisation du comportement de la validation croisée. Notez que [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) n'est pas affecté par les classes ou les groupes.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png)

Chaque pli est constitué de deux tableaux : le premier est lié à l'_ensemble d'entraînement_, et le second à l'_ensemble de test_. Ainsi, on peut créer les ensembles d'entraînement/test en utilisant l'indexation numpy :

In [3]:
X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]

#### <a id='repeated-k-fold'></a> K-fold répétée

[**`RepeatedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedKFold.html) répète [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) $n$ fois. Il peut être utilisé lorsque l'on souhaite exécuter KFold $n$ fois, en produisant différentes divisions à chaque répétition.

Exemple de validation croisée à 2 plis répétée 2 fois :

In [5]:
import numpy as np
from sklearn.model_selection import RepeatedKFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
random_state = 12883823
rkf = RepeatedKFold(n_splits=2, n_repeats=2, random_state=random_state)
for train, test in rkf.split(X):
    print(f"{train} {test}")
# [2 3] [0 1]
# [0 1] [2 3]
# [0 2] [1 3]
# [1 3] [0 2]

[2 3] [0 1]
[0 1] [2 3]
[0 2] [1 3]
[1 3] [0 2]


De même, [**`RepeatedStratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html) répète Stratified K-Fold $n$ fois avec une randomisation différente à chaque répétition.

#### <a id='leave-one-out-loo'></a> Leave One Out (LOO)

[**`LeaveOneOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html) (ou LOO) est une validation croisée simple. Chaque ensemble d'apprentissage est créé en prenant tous les échantillons sauf un, l'ensemble de test étant l'échantillon exclu. Ainsi, pour $n$ échantillons, nous avons $n$ ensembles d'apprentissage différents et $n$ ensembles de test différents. Cette procédure de validation croisée ne gaspille pas beaucoup de données car un seul échantillon est retiré de l'ensemble d'apprentissage :

In [6]:
from sklearn.model_selection import LeaveOneOut

X = [1, 2, 3, 4]
loo = LeaveOneOut()
for train, test in loo.split(X):
    print(f"{train} {test}")
# [1 2 3] [0]
# [0 2 3] [1]
# [0 1 3] [2]
# [0 1 2] [3]

[1 2 3] [0]
[0 2 3] [1]
[0 1 3] [2]
[0 1 2] [3]


Les utilisateurs potentiels de LOO pour la sélection de modèles doivent prendre en compte quelques avertissements connus. Par rapport à la validation croisée $k$-fold, on construit $n$ modèles à partir des échantillons au lieu de $k$ modèles, où $n \gt k$. De plus, chaque modèle est entraîné sur $n - 1$ échantillons au lieu de $(k-1) n / k$. Dans les deux cas, en supposant que $k$ n'est pas trop grand et $k \lt n$, LOO est plus coûteux en termes de calcul que la validation croisée $k$-fold.

En termes de précision, LOO conduit souvent à une grande variance en tant qu'estimateur de l'erreur de test. Intuitivement, étant donné que $n - 1$ des $n$ échantillons sont utilisés pour construire chaque modèle, les modèles construits à partir des plis sont pratiquement identiques les uns aux autres et au modèle construit à partir de l'ensemble d'apprentissage complet.

Cependant, si la courbe d'apprentissage est raide pour la taille d'entraînement considérée, une validation croisée à 5 ou 10 plis peut surestimer l'erreur de généralisation.

En règle générale, la plupart des auteurs et des preuves empiriques suggèrent que la validation croisée à 5 ou 10 plis devrait être préférée à LOO.

##### Références

- http://www.faqs.org/faqs/ai-faq/neural-nets/part3/section-12.html
- 📚 T. Hastie, R. Tibshirani et J. Friedman, [**“Elements of Statistical Learning Ed. 2”**](https://hastie.su.domains/Papers/ESLII.pdf), Springer, 2009.
- 🔬 L. Breiman, P. Spector [**“Submodel selection and evaluation in regression: The X-random case”**](https://digitalassets.lib.berkeley.edu/sdtr/ucb/text/197.pdf), International Statistical Review 1992.
- 🔬 R. Kohavi, [**“A Study of Cross-Validation and Bootstrap for Accuracy Estimation and Model Selection”**](http://ai.stanford.edu/~ronnyk/accEst.pdf), Intl. Jnt. Conf. AI.
- 🔬 R. Bharat Rao, G. Fung, R. Rosales, [**“On the Dangers of Cross-Validation. An Experimental Evaluation”**](https://people.csail.mit.edu/romer/papers/CrossVal_SDM08.pdf), SIAM 2008.
- 📚 G. James, D. Witten, T. Hastie, R Tibshirani, [**“An Introduction to Statistical Learning”**](https://hastie.su.domains/ISLR2/ISLRv2_website.pdf), Springer 2013.

#### <a id='leave-p-out-lpo'></a> Leave P Out (LPO)

[**`LeavePOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePOut.html) est très similaire à [**`LeaveOneOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html) car il crée tous les ensembles d'entraînement/test possibles en supprimant $p$ échantillons de l'ensemble complet. Pour $n$ échantillons, cela produit ${n \choose p}$ paires d'ensembles d'entraînement/test. Contrairement à [**`LeaveOneOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneOut.html) et [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), les ensembles de test se chevaucheront pour $p \gt 1$.

Exemple de Leave-2-Out sur un ensemble de données avec 4 échantillons :

In [4]:
from sklearn.model_selection import LeavePOut

X = np.ones(4)
lpo = LeavePOut(p=2)
for train, test in lpo.split(X):
    print(f"{train} {test}")
# [2 3] [0 1]
# [1 3] [0 2]
# [1 2] [0 3]
# [0 3] [1 2]
# [0 2] [1 3]
# [0 1] [2 3]

[2 3] [0 1]
[1 3] [0 2]
[1 2] [0 3]
[0 3] [1 2]
[0 2] [1 3]
[0 1] [2 3]


#### <a id='random-permutations-cross-validation-a-k-a-shuffle-split'></a> Validation croisée par permutations aléatoires, alias Shuffle & Split

L'itérateur [**`ShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) générera un nombre défini par l'utilisateur de jeux de données d'entraînement/test indépendants. Les échantillons sont d'abord mélangés, puis divisés en une paire d'ensembles d'entraînement et de test.

Il est possible de contrôler l'aléatoire pour assurer la reproductibilité des résultats en initialisant explicitement le générateur de nombres pseudo-aléatoires `random_state`.

Voici un exemple d'utilisation :

In [7]:
from sklearn.model_selection import ShuffleSplit
X = np.arange(10)
ss = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
for train_index, test_index in ss.split(X):
    print(f"{train_index} {test_index}")
# [9 1 6 7 3 0 5] [2 8 4]
# [2 9 8 0 6 7 4] [3 5 1]
# [4 5 1 0 6 9 7] [2 3 8]
# [2 7 5 8 0 3 4] [6 1 9]
# [4 1 0 6 8 9 3] [5 2 7]

[9 1 6 7 3 0 5] [2 8 4]
[2 9 8 0 6 7 4] [3 5 1]
[4 5 1 0 6 9 7] [2 3 8]
[2 7 5 8 0 3 4] [6 1 9]
[4 1 0 6 8 9 3] [5 2 7]


Voici une visualisation du comportement de la validation croisée. Notez que [**`ShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) n'est pas influencé par les classes ou les groupes.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_008.png)

[**`ShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) est donc une bonne alternative à la validation croisée [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) qui permet un contrôle plus précis du nombre d'itérations et de la proportion d'échantillons de chaque côté de la division d'entraînement/test.

### <a id='cross-validation-iterators-with-stratification-based-on-class-labels'></a> 3.1.2.2. Itérateurs de validation croisée avec stratification basée sur les étiquettes de classe

Certains problèmes de classification peuvent présenter un déséquilibre important dans la distribution des classes cibles : par exemple, il peut y avoir plusieurs fois plus d'échantillons négatifs que d'échantillons positifs. Dans de tels cas, il est recommandé d'utiliser un échantillonnage stratifié tel qu'implémenté dans [**`StratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) et [**`StratifiedShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) pour garantir que les fréquences relatives des classes sont approximativement préservées dans chaque pli d'entraînement et de validation.

#### <a id='stratified-k-fold'></a> K-fold stratifié

[**`StratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) est une variation de la validation croisée _k-fold_ qui renvoie des plis _stratifiés_ : chaque ensemble contient approximativement le même pourcentage d'échantillons de chaque classe cible que l'ensemble complet.

Voici un exemple de validation croisée stratifiée à 3 plis sur un ensemble de données avec 50 échantillons provenant de deux classes déséquilibrées. Nous montrons le nombre d'échantillons dans chaque classe et comparons avec [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).

In [8]:
from sklearn.model_selection import StratifiedKFold, KFold
import numpy as np
X, y = np.ones((50, 1)), np.hstack(([0] * 45, [1] * 5))
skf = StratifiedKFold(n_splits=3)
for train, test in skf.split(X, y):
    print(f'train -  {np.bincount(y[train])}   |   test -  {np.bincount(y[test])}')
# train -  [30  3]   |   test -  [15  2]
# train -  [30  3]   |   test -  [15  2]
# train -  [30  4]   |   test -  [15  1]
kf = KFold(n_splits=3)
for train, test in kf.split(X, y):
    print(f'train -  {np.bincount(y[train])}   |   test -  {np.bincount(y[test])}')
# train -  [28  5]   |   test -  [17]
# train -  [28  5]   |   test -  [17]
# train -  [34]   |   test -  [11  5]

train -  [30  3]   |   test -  [15  2]
train -  [30  3]   |   test -  [15  2]
train -  [30  4]   |   test -  [15  1]
train -  [28  5]   |   test -  [17]
train -  [28  5]   |   test -  [17]
train -  [34]   |   test -  [11  5]


Nous pouvons voir que [**`StratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) préserve les ratios de classes (environ 1/10) à la fois dans l'ensemble d'apprentissage et dans l'ensemble de test.

Voici une visualisation du comportement de la validation croisée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_009.png)

#### <a id='stratified-shuffle-split'></a> Stratified Shuffle Split

[**`StratifiedShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedShuffleSplit.html) est une variation de _ShuffleSplit_ qui renvoie des séparations stratifiées, c'est-à-dire qui crée des séparations en conservant le même pourcentage pour chaque classe cible que dans l'ensemble complet.

Voici une visualisation du comportement de la validation croisée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_012.png)

### <a id='cross-validation-iterators-for-grouped-data'></a> 3.1.2.3. Itérateurs de validation croisée pour les données regroupées

L'hypothèse i.i.d. est rompue si le processus génératif sous-jacent produit des groupes d'échantillons dépendants.

Un tel regroupement de données dépend du domaine. Par exemple, il peut y avoir des données médicales collectées auprès de plusieurs patients, avec plusieurs échantillons prélevés sur chaque patient. Et de telles données sont susceptibles de dépendre du groupe individuel. Dans notre exemple, l'identifiant du patient pour chaque échantillon sera son identifiant de groupe.

Dans ce cas, nous aimerions savoir si un modèle entraîné sur un ensemble particulier de groupes généralise bien aux groupes non vus. Pour mesurer cela, nous devons nous assurer que tous les échantillons dans le pli de validation proviennent de groupes qui ne sont pas du tout représentés dans le pli d'apprentissage associé.

Les diviseurs de validation croisée suivants peuvent être utilisés pour cela. L'identifiant de regroupement des échantillons est spécifié via le paramètre `groups`.

#### <a id='group-k-fold'></a> K-fold par groupe

[**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) est une variation de la validation croisée _k-fold_ qui garantit que le même groupe n'est pas représenté à la fois dans les ensembles de test et d'apprentissage. Par exemple, si les données sont obtenues à partir de sujets différents avec plusieurs échantillons par sujet et si le modèle est suffisamment flexible pour apprendre à partir de caractéristiques très spécifiques à chaque personne, il pourrait ne pas généraliser à de nouveaux sujets. [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) permet de détecter ce type de situations de surajustement.

Imaginez que vous ayez trois sujets, chacun avec un numéro associé de 1 à 3 :

In [9]:
from sklearn.model_selection import GroupKFold

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 8.8, 9, 10]
y = ["a", "b", "b", "b", "c", "c", "c", "d", "d", "d"]
groups = [1, 1, 1, 2, 2, 2, 3, 3, 3, 3]

gkf = GroupKFold(n_splits=3)
for train, test in gkf.split(X, y, groups=groups):
    print(f"{train} {test}")
# [0 1 2 3 4 5] [6 7 8 9]
# [0 1 2 6 7 8 9] [3 4 5]
# [3 4 5 6 7 8 9] [0 1 2]

[0 1 2 3 4 5] [6 7 8 9]
[0 1 2 6 7 8 9] [3 4 5]
[3 4 5 6 7 8 9] [0 1 2]


Chaque sujet se trouve dans un pli de test différent, et le même sujet n'est jamais à la fois dans le test et dans l'apprentissage. Notez que les plis n'ont pas exactement la même taille en raison du déséquilibre dans les données. Si les proportions de classe doivent être équilibrées dans les plis, [**`StratifiedGroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedGroupKFold.html) est une meilleure option.

Voici une visualisation du comportement de la validation croisée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_007.png)

Similaire à [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), les ensembles de test de [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) formeront une partition complète de l'ensemble des données. Contrairement à [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) n'est pas du tout aléatoire, tandis que KFold est aléatoire lorsque `shuffle=True`.

#### <a id='stratifiedgroupkfold'></a> StratifiedGroupKFold

[**`StratifiedGroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedGroupKFold.html) est un schéma de validation croisée qui combine à la fois [**`StratifiedKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) et [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html). L'idée est de tenter de préserver la distribution des classes dans chaque séparation tout en maintenant chaque groupe dans une seule séparation. Cela peut être utile lorsque vous disposez d'un ensemble de données déséquilibré, de sorte que l'utilisation uniquement de [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) pourrait produire des séparations biaisées.

Exemple :

In [10]:
from sklearn.model_selection import StratifiedGroupKFold
X = list(range(18))
y = [1] * 6 + [0] * 12
groups = [1, 2, 3, 3, 4, 4, 1, 1, 2, 2, 3, 4, 5, 5, 5, 6, 6, 6]
sgkf = StratifiedGroupKFold(n_splits=3)
for train, test in sgkf.split(X, y, groups=groups):
    print(f"{train} {test}")
# [ 0  2  3  4  5  6  7 10 11 15 16 17] [ 1  8  9 12 13 14]
# [ 0  1  4  5  6  7  8  9 11 12 13 14] [ 2  3 10 15 16 17]
# [ 1  2  3  8  9 10 12 13 14 15 16 17] [ 0  4  5  6  7 11]

[ 0  2  3  4  5  6  7 10 11 15 16 17] [ 1  8  9 12 13 14]
[ 0  1  4  5  6  7  8  9 11 12 13 14] [ 2  3 10 15 16 17]
[ 1  2  3  8  9 10 12 13 14 15 16 17] [ 0  4  5  6  7 11]


Notes d'implémentation :

- Avec l'implémentation actuelle, un mélange complet n'est pas possible dans la plupart des scénarios. Lorsque `shuffle=True`, les étapes suivantes se produisent :
    1. Tous les groupes sont mélangés.
    2. Les groupes sont triés par écart type des classes en utilisant un tri stable.
    3. Les groupes triés sont parcourus et assignés aux plis.
- Cela signifie que seuls les groupes ayant le même écart type de distribution de classes seront mélangés, ce qui peut être utile lorsque chaque groupe n'a qu'une seule classe.
- L'algorithme attribue de manière gloutonne chaque groupe à un des `n_splits` ensembles de test, en choisissant l'ensemble de test qui minimise la variance de la distribution des classes entre les ensembles de test. L'assignation des groupes se fait des groupes avec la plus grande variance à la plus faible variance en termes de fréquence des classes, c'est-à-dire que les grands groupes ayant des valeurs concentrées sur une ou quelques classes sont assignés en premier.
- Cette division est sous-optimale dans le sens où elle peut produire des séparations déséquilibrées même si une stratification parfaite est possible. Si vous avez une répartition relativement proche des classes dans chaque groupe, il vaut mieux utiliser [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html).

Voici une visualisation du comportement de la validation croisée pour des groupes inégaux :

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_005.png)

#### <a id='leave-one-group-out'></a> Laisser un groupe de côté (LOGO - _Leave One Group Out_)

[**`LeaveOneGroupOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html) est un schéma de validation croisée dans lequel chaque division exclut les échantillons appartenant à un groupe spécifique. Les informations de groupe sont fournies via un tableau qui encode le groupe de chaque échantillon.

Chaque ensemble d'entraînement est donc constitué de tous les échantillons, sauf ceux d'un groupe spécifique. Cela correspond à [**`LeavePGroupsOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePGroupsOut.html) avec `n_groups=1` et est similaire à [**`GroupKFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupKFold.html) avec `n_splits` égal au nombre d'étiquettes uniques passées au paramètre `groups`.

Par exemple, dans le cas d'expériences multiples, [**`LeaveOneGroupOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html) peut être utilisé pour créer une validation croisée basée sur différentes expériences : nous créons un ensemble d'entraînement en utilisant les échantillons de toutes les expériences, à l'exception d'une seule d'entre elles :

In [ ]:
from sklearn.model_selection import LeaveOneGroupOut

X = [1, 5, 10, 50, 60, 70, 80]
y = [0, 1, 1, 2, 2, 2, 2]
groups = [1, 1, 2, 2, 3, 3, 3]
logo = LeaveOneGroupOut()
for train, test in logo.split(X, y, groups=groups):
    print(f"{train} {test}")
# [2 3 4 5 6] [0 1]
# [0 1 4 5 6] [2 3]
# [0 1 2 3] [4 5 6]

Une autre application courante est d'utiliser des informations temporelles : par exemple, les groupes pourraient être l'année de collecte des échantillons, permettant ainsi une validation croisée basée sur des séparations temporelles.

#### <a id='leave-p-groups-out'></a> Laisser P groupes de côté (LPGO - _Leave P Groups Out_)

[**`LeavePGroupsOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePGroupsOut.html) est similaire à [**`LeaveOneGroupOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeaveOneGroupOut.html), mais il supprime les échantillons liés à $P$ groupes pour chaque ensemble d'entraînement/test. Toutes les combinaisons possibles de $P$ groupes sont exclues, ce qui signifie que les ensembles de test se chevauchent lorsque $P \gt 1$.

Exemple de Laisser-2-Groupes de côté :

In [ ]:
from sklearn.model_selection import LeavePGroupsOut

X = np.arange(6)
y = [1, 1, 1, 2, 2, 2]
groups = [1, 1, 2, 2, 3, 3]
lpgo = LeavePGroupsOut(n_groups=2)
for train, test in lpgo.split(X, y, groups=groups):
    print(f"{train} {test}")
# [4 5] [0 1 2 3]
# [2 3] [0 1 4 5]
# [0 1] [2 3 4 5]

#### <a id='group-shuffle-split'></a> Groupe Shuffle Split

L'itérateur [**`GroupShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) fonctionne comme une combinaison de [**`ShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) et de [**`LeavePGroupsOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePGroupsOut.html), et génère une séquence de partitions aléatoires dans lesquelles un sous-ensemble de groupes est exclu pour chaque division. Chaque division d'entraînement/test est effectuée indépendamment, ce qui signifie qu'il n'y a pas de relation garantie entre les ensembles de test successifs.

Voici un exemple d'utilisation :

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = [0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 0.001]
y = ["a", "b", "b", "b", "c", "c", "c", "a"]
groups = [1, 1, 2, 2, 3, 3, 4, 4]
gss = GroupShuffleSplit(n_splits=4, test_size=0.5, random_state=0)
for train, test in gss.split(X, y, groups=groups):
    print(f"{train} {test}")
# [0 1 2 3] [4 5 6 7]
# [2 3 6 7] [0 1 4 5]
# [2 3 4 5] [0 1 6 7]
# [4 5 6 7] [0 1 2 3]

Voici une visualisation du comportement de la validation croisée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_011.png)

Cette classe est utile lorsque le comportement de [**`LeavePGroupsOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePGroupsOut.html) est souhaité, mais que le nombre de groupes est suffisamment élevé pour que la génération de toutes les partitions possibles avec $P$ groupes exclus soit prohibitivement coûteuse. Dans un tel scénario, [**`GroupShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GroupShuffleSplit.html) fournit un échantillon aléatoire (avec remplacement) des ensembles d'entraînement/test générés par [**`LeavePGroupsOut`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.LeavePGroupsOut.html).

### <a id='predefined-fold-splits-validation-sets'></a> 3.1.2.4. Divisions prédéfinies

Pour certains ensembles de données, une répartition prédéfinie des données en ensembles d'entraînement et de validation, ou en plusieurs ensembles de validation croisée, existe déjà. En utilisant [**`PredefinedSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.PredefinedSplit.html), il est possible d'utiliser ces répartitions, par exemple lors de la recherche des hyperparamètres.

Par exemple, lorsque vous utilisez un ensemble de validation, définissez la valeur `test_fold` à 0 pour tous les échantillons faisant partie de l'ensemble de validation, et à -1 pour tous les autres échantillons.

### <a id='using-cross-validation-iterators-to-split-train-and-test'></a> 3.1.2.5. Utilisation d'itérateurs de validation croisée pour séparer les ensembles d'entraînement et de test

Les fonctions de validation croisée basées sur les groupes mentionnées ci-dessus peuvent également être utiles pour diviser un ensemble de données en sous-ensembles d'entraînement et de test. Notez que la fonction pratique [**`train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) est un wrapper autour de [**`ShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) et ne permet donc que des divisions stratifiées (en utilisant les étiquettes de classe) et ne prend pas en compte les groupes.

Pour effectuer la division en ensembles d'entraînement et de test, utilisez les indices des sous-ensembles d'entraînement et de test générés par le générateur renvoyé par la méthode `split()` du diviseur de validation croisée. Par exemple :

In [ ]:
import numpy as np
from sklearn.model_selection import GroupShuffleSplit

X = np.array([0.1, 0.2, 2.2, 2.4, 2.3, 4.55, 5.8, 0.001])
y = np.array(["a", "b", "b", "b", "c", "c", "c", "a"])
groups = np.array([1, 1, 2, 2, 3, 3, 4, 4])
train_indx, test_indx = next(
    GroupShuffleSplit(random_state=7).split(X, y, groups)
)
X_train, X_test, y_train, y_test = \
    X[train_indx], X[test_indx], y[train_indx], y[test_indx]
X_train.shape, X_test.shape
# ((6,), (2,))
np.unique(groups[train_indx]), np.unique(groups[test_indx])
# (array([1, 2, 4]), array([3]))

### <a id='cross-validation-of-time-series-data'></a> 3.1.2.6. Validation croisée des données de séries temporelles

Les données de séries temporelles se caractérisent par la corrélation entre les observations qui sont proches dans le temps (_autocorrélation_). Cependant, les techniques de validation croisée classiques telles que [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) et [**`ShuffleSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html) supposent que les échantillons sont indépendants et identiquement distribués, ce qui conduirait à une corrélation déraisonnable entre les instances d'entraînement et de test (donnant de mauvaises estimations de l'erreur de généralisation) sur les données de séries temporelles. Par conséquent, il est très important d'évaluer notre modèle pour les données de séries temporelles sur les observations "futures" les moins similaires à celles utilisées pour entraîner le modèle. Pour cela, une solution est proposée par [**`TimeSeriesSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html).

#### <a id='time-series-split'></a> Division des séries temporelles

[**`TimeSeriesSplit`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html) est une variation de _k-fold_ qui renvoie les $k$ premières partitions comme ensemble d'entraînement et la $k + 1$-ème partition comme ensemble de test. Notez que, contrairement aux méthodes de validation croisée standard, les ensembles d'entraînement successifs sont des ensembles englobants de ceux qui les précèdent. De plus, tous les échantillons excédentaires sont ajoutés à la première partition d'entraînement, qui est toujours utilisée pour entraîner le modèle.

Cette classe peut être utilisée pour effectuer la validation croisée des échantillons de données de séries temporelles observées à intervalles de temps fixes.

Exemple d'une validation croisée en 3 parties sur des données de séries temporelles avec 6 échantillons :

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit(n_splits=3)
print(tscv)
TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None)
for train, test in tscv.split(X):
    print(f"{train} {test}")
# [0 1 2] [3]
# [0 1 2 3] [4]
# [0 1 2 3 4] [5]

Voici une visualisation du comportement de la validation croisée.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)

## <a id='a-note-on-shuffling'></a> 3.1.3. Note sur le mélange

Si l'ordre des données n'est pas arbitraire (par exemple, les échantillons avec la même étiquette de classe sont contigus), le mélanger en premier peut être essentiel pour obtenir un résultat de validation croisée significatif. Cependant, l'inverse peut être vrai si les échantillons ne sont pas indépendants et identiquement distribués. Par exemple, si les échantillons correspondent à des articles de presse et sont triés par leur date de publication, alors le mélange des données conduira probablement à un modèle qui est surajusté et à un score de validation artificiellement élevé : il sera testé sur des échantillons artificiellement similaires (proches dans le temps) aux échantillons d'entraînement.

Certains diviseurs de validation croisée, comme [**`KFold`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html), ont une option intégrée pour mélanger les indices des données avant de les diviser. Notez que :

- Cela consomme moins de mémoire que le mélange direct des données.
- Par défaut, aucun mélange n'est effectué, y compris pour la validation croisée (stratifiée) K-fold réalisée en spécifiant `cv=un_entier` à [**`cross_val_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html), la recherche en grille, etc. Gardez à l'esprit que [**`train_test_split`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) retourne toujours une division aléatoire.
- Le paramètre `random_state` est par défaut à `None`, ce qui signifie que le mélange sera différent à chaque itération de `KFold(..., shuffle=True)`. Cependant, `GridSearchCV` utilisera le même mélange pour chaque ensemble de paramètres validé par un seul appel à sa méthode `fit`.
- Pour obtenir des résultats identiques pour chaque division, définissez `random_state` sur un entier.

Pour plus de détails sur la façon de contrôler le caractère aléatoire des diviseurs de validation croisée et éviter les problèmes courants, voir [**Contrôler le hasard** (10.3)](https://scikit-learn.org/stable/common_pitfalls.html#randomness).

## <a id='cross-validation-and-model-selection'></a> 3.1.4. Validation croisée et sélection de modèle

Les diviseurs de validation croisée peuvent également être utilisés pour effectuer directement la sélection de modèle en utilisant la recherche en grille pour les hyperparamètres optimaux du modèle. C'est le sujet de la prochaine section : [**Réglage des hyperparamètres d'un estimateur** (3.2)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search).

## <a id='permutation-test-score'></a> 3.1.5. Score de test de permutation

[**`permutation_test_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.permutation_test_score.html) offre une autre façon d'évaluer les performances des classifieurs. Il fournit une valeur p basée sur des permutations, qui représente à quel point les performances observées du classifieur pourraient être obtenues par hasard. L'hypothèse nulle dans ce test est que le classifieur ne parvient pas à exploiter une quelconque dépendance statistique entre les caractéristiques et les étiquettes pour effectuer des prédictions correctes sur les données exclues. [**`permutation_test_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.permutation_test_score.html) génère une distribution nulle en calculant `n_permutations` différentes permutations des données. Dans chaque permutation, les étiquettes sont mélangées aléatoirement, supprimant ainsi toute dépendance entre les caractéristiques et les étiquettes. La valeur p est le nombre de permutations pour lesquelles le score de validation croisée moyen obtenu par le modèle est meilleur que le score de validation croisée obtenu par le modèle en utilisant les données d'origine. Pour des résultats fiables, `n_permutations` devrait généralement être supérieur à 100 et `cv` entre 3 et 10 partitions.

Une faible valeur p indique que l'ensemble de données contient une dépendance réelle entre les caractéristiques et les étiquettes et que le classifieur a été capable de l'utiliser pour obtenir de bons résultats. Une valeur p élevée peut être due à l'absence de dépendance entre les caractéristiques et les étiquettes (il n'y a pas de différence dans les valeurs des caractéristiques entre les classes) ou parce que le classifieur n'a pas pu utiliser la dépendance dans les données. Dans ce dernier cas, l'utilisation d'un classifieur plus approprié capable d'utiliser la structure des données conduirait à une valeur p plus faible.

La validation croisée fournit des informations sur la capacité de généralisation d'un classifieur, en particulier sur la plage des erreurs attendues du classifieur. Cependant, un classifieur entraîné sur un ensemble de données de grande dimension sans structure peut encore mieux se comporter que prévu en validation croisée, simplement par hasard. Cela peut se produire généralement avec de petits ensembles de données contenant moins de quelques centaines d'échantillons. [**`permutation_test_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.permutation_test_score.html) fournit des informations sur la mesure dans laquelle le classifieur a trouvé une véritable structure de classe et peut aider à évaluer les performances du classifieur.

Il est important de noter que ce test a montré qu'il produit des valeurs p basses même s'il n'y a qu'une faible structure dans les données, car il n'y a absolument aucune structure dans les ensembles de données permutés correspondants. Ce test est donc capable de montrer quand le modèle surpasse de manière fiable une prédiction aléatoire.

Enfin, [**`permutation_test_score`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.permutation_test_score.html) est calculé de manière exhaustive et ajuste en interne `(n_permutations + 1) * n_cv` modèles. Il est donc réalisable uniquement avec de petits ensembles de données pour lesquels l'ajustement d'un modèle individuel est très rapide.

### Exemples

#### [**Test par permutations de la signifiance d'un score de classification**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_permutation_tests_for_classification.ipynb)<br/>([_Test with permutations the significance of a classification score_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_permutation_tests_for_classification.html))